<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#2018-12-18" data-toc-modified-id="2018-12-18-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2018-12-18</a></span><ul class="toc-item"><li><span><a href="#検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ" data-toc-modified-id="検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ</a></span><ul class="toc-item"><li><span><a href="#登場キーワードについての調査" data-toc-modified-id="登場キーワードについての調査-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>登場キーワードについての調査</a></span><ul class="toc-item"><li><span><a href="#DRONE-:-CI/CD-のパイプライン処理" data-toc-modified-id="DRONE-:-CI/CD-のパイプライン処理-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>DRONE : CI/CD のパイプライン処理</a></span></li><li><span><a href="#DigDag-:-機械学習処理システム向けのパイプライン処理ツール" data-toc-modified-id="DigDag-:-機械学習処理システム向けのパイプライン処理ツール-2.1.1.2"><span class="toc-item-num">2.1.1.2&nbsp;&nbsp;</span>DigDag : 機械学習処理システム向けのパイプライン処理ツール</a></span></li><li><span><a href="#.dig-ファイルのサンプル集" data-toc-modified-id=".dig-ファイルのサンプル集-2.1.1.3"><span class="toc-item-num">2.1.1.3&nbsp;&nbsp;</span>.dig ファイルのサンプル集</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#2018-12-19" data-toc-modified-id="2018-12-19-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2018-12-19</a></span><ul class="toc-item"><li><span><a href="#機械学習プロジェクトが失敗する9つの理由---六本木で働くデータサイエンティストのブログ" data-toc-modified-id="機械学習プロジェクトが失敗する9つの理由---六本木で働くデータサイエンティストのブログ-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>機械学習プロジェクトが失敗する9つの理由 - 六本木で働くデータサイエンティストのブログ</a></span><ul class="toc-item"><li><span><a href="#ML-Ops-のポイント" data-toc-modified-id="ML-Ops-のポイント-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>ML Ops のポイント</a></span></li><li><span><a href="#機械学習プロジェクトが失敗する理由" data-toc-modified-id="機械学習プロジェクトが失敗する理由-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>機械学習プロジェクトが失敗する理由</a></span><ul class="toc-item"><li><span><a href="#課題設定の誤り" data-toc-modified-id="課題設定の誤り-3.1.2.1"><span class="toc-item-num">3.1.2.1&nbsp;&nbsp;</span>課題設定の誤り</a></span></li><li><span><a href="#間違った問題に機械学習を使っている" data-toc-modified-id="間違った問題に機械学習を使っている-3.1.2.2"><span class="toc-item-num">3.1.2.2&nbsp;&nbsp;</span>間違った問題に機械学習を使っている</a></span></li><li><span><a href="#十分なデータがない" data-toc-modified-id="十分なデータがない-3.1.2.3"><span class="toc-item-num">3.1.2.3&nbsp;&nbsp;</span>十分なデータがない</a></span></li><li><span><a href="#適切なデータがない（データのクオリティが低い）" data-toc-modified-id="適切なデータがない（データのクオリティが低い）-3.1.2.4"><span class="toc-item-num">3.1.2.4&nbsp;&nbsp;</span>適切なデータがない（データのクオリティが低い）</a></span></li><li><span><a href="#(不要な）データが多すぎる" data-toc-modified-id="(不要な）データが多すぎる-3.1.2.5"><span class="toc-item-num">3.1.2.5&nbsp;&nbsp;</span>(不要な）データが多すぎる</a></span></li><li><span><a href="#人材の配置が適材適所ではない" data-toc-modified-id="人材の配置が適材適所ではない-3.1.2.6"><span class="toc-item-num">3.1.2.6&nbsp;&nbsp;</span>人材の配置が適材適所ではない</a></span></li><li><span><a href="#間違ったツールを使っている" data-toc-modified-id="間違ったツールを使っている-3.1.2.7"><span class="toc-item-num">3.1.2.7&nbsp;&nbsp;</span>間違ったツールを使っている</a></span></li><li><span><a href="#適切なモデルを使っていない" data-toc-modified-id="適切なモデルを使っていない-3.1.2.8"><span class="toc-item-num">3.1.2.8&nbsp;&nbsp;</span>適切なモデルを使っていない</a></span></li><li><span><a href="#適切な評価指標を使っていない" data-toc-modified-id="適切な評価指標を使っていない-3.1.2.9"><span class="toc-item-num">3.1.2.9&nbsp;&nbsp;</span>適切な評価指標を使っていない</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#2018-12-25" data-toc-modified-id="2018-12-25-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>2018-12-25</a></span><ul class="toc-item"><li><span><a href="#データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由" data-toc-modified-id="データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由</a></span><ul class="toc-item"><li><span><a href="#自分のキャリアと会社を切り離して考えるべき時代" data-toc-modified-id="自分のキャリアと会社を切り離して考えるべき時代-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>自分のキャリアと会社を切り離して考えるべき時代</a></span></li><li><span><a href="#データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う" data-toc-modified-id="データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う</a></span><ul class="toc-item"><li><span><a href="#データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）" data-toc-modified-id="データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）-4.1.2.1"><span class="toc-item-num">4.1.2.1&nbsp;&nbsp;</span>データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）</a></span></li><li><span><a href="#なんちゃってデータサイエンティスト" data-toc-modified-id="なんちゃってデータサイエンティスト-4.1.2.2"><span class="toc-item-num">4.1.2.2&nbsp;&nbsp;</span>なんちゃってデータサイエンティスト</a></span></li><li><span><a href="#Excelに固執せず、スケールするテクノロジーを" data-toc-modified-id="Excelに固執せず、スケールするテクノロジーを-4.1.2.3"><span class="toc-item-num">4.1.2.3&nbsp;&nbsp;</span>Excelに固執せず、スケールするテクノロジーを</a></span></li></ul></li><li><span><a href="#データサイエンス・機械学習導入の意義" data-toc-modified-id="データサイエンス・機械学習導入の意義-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>データサイエンス・機械学習導入の意義</a></span></li><li><span><a href="#時代の流れに反しない仕事のやり方を" data-toc-modified-id="時代の流れに反しない仕事のやり方を-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>時代の流れに反しない仕事のやり方を</a></span></li><li><span><a href="#メモ" data-toc-modified-id="メモ-4.1.5"><span class="toc-item-num">4.1.5&nbsp;&nbsp;</span>メモ</a></span></li></ul></li></ul></li><li><span><a href="#2018-12-28" data-toc-modified-id="2018-12-28-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>2018-12-28</a></span><ul class="toc-item"><li><span><a href="#年末に勉強しておきたいAIのAPI" data-toc-modified-id="年末に勉強しておきたいAIのAPI-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>年末に勉強しておきたいAIのAPI</a></span></li><li><span><a href="#最先端と現場の狭間で" data-toc-modified-id="最先端と現場の狭間で-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>最先端と現場の狭間で</a></span><ul class="toc-item"><li><span><a href="#怒涛の勢いで発達する最先端" data-toc-modified-id="怒涛の勢いで発達する最先端-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>怒涛の勢いで発達する最先端</a></span></li><li><span><a href="#機械学習の現場" data-toc-modified-id="機械学習の現場-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>機械学習の現場</a></span></li></ul></li></ul></li><li><span><a href="#2018-12-30" data-toc-modified-id="2018-12-30-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>2018-12-30</a></span><ul class="toc-item"><li><span><a href="#便利なリンク集" data-toc-modified-id="便利なリンク集-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>便利なリンク集</a></span><ul class="toc-item"><li><span><a href="#統計用語集" data-toc-modified-id="統計用語集-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>統計用語集</a></span></li><li><span><a href="#確率分布一覧" data-toc-modified-id="確率分布一覧-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>確率分布一覧</a></span><ul class="toc-item"><li><span><a href="#ガンマ分布" data-toc-modified-id="ガンマ分布-6.1.2.1"><span class="toc-item-num">6.1.2.1&nbsp;&nbsp;</span>ガンマ分布</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#2018-12-31" data-toc-modified-id="2018-12-31-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>2018-12-31</a></span><ul class="toc-item"><li><span><a href="#2019年のスタートアップを占う" data-toc-modified-id="2019年のスタートアップを占う-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>2019年のスタートアップを占う</a></span></li></ul></li><li><span><a href="#2019-01-07" data-toc-modified-id="2019-01-07-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>2019-01-07</a></span><ul class="toc-item"><li><span><a href="#おさらい：昨年話題になったテクノロジートピック@機械学習分野" data-toc-modified-id="おさらい：昨年話題になったテクノロジートピック@機械学習分野-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>おさらい：昨年話題になったテクノロジートピック@機械学習分野</a></span><ul class="toc-item"><li><span><a href="#Google-Research-BERT" data-toc-modified-id="Google-Research-BERT-8.1.1"><span class="toc-item-num">8.1.1&nbsp;&nbsp;</span>Google Research BERT</a></span><ul class="toc-item"><li><span><a href="#BERT関連情報リソース" data-toc-modified-id="BERT関連情報リソース-8.1.1.1"><span class="toc-item-num">8.1.1.1&nbsp;&nbsp;</span>BERT関連情報リソース</a></span></li></ul></li><li><span><a href="#Deep-Learningはなぜ大域最適解にたどり着けるのか" data-toc-modified-id="Deep-Learningはなぜ大域最適解にたどり着けるのか-8.1.2"><span class="toc-item-num">8.1.2&nbsp;&nbsp;</span>Deep Learningはなぜ大域最適解にたどり着けるのか</a></span></li><li><span><a href="#AutoML" data-toc-modified-id="AutoML-8.1.3"><span class="toc-item-num">8.1.3&nbsp;&nbsp;</span>AutoML</a></span><ul class="toc-item"><li><span><a href="#ポイント：機械学習システムのマッシュアップ？" data-toc-modified-id="ポイント：機械学習システムのマッシュアップ？-8.1.3.1"><span class="toc-item-num">8.1.3.1&nbsp;&nbsp;</span>ポイント：機械学習システムのマッシュアップ？</a></span></li></ul></li><li><span><a href="#転移学習の使いどころ" data-toc-modified-id="転移学習の使いどころ-8.1.4"><span class="toc-item-num">8.1.4&nbsp;&nbsp;</span>転移学習の使いどころ</a></span></li></ul></li><li><span><a href="#DeNAのKaggle制度" data-toc-modified-id="DeNAのKaggle制度-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>DeNAのKaggle制度</a></span></li><li><span><a href="#機械学習の普及が加速する2019年" data-toc-modified-id="機械学習の普及が加速する2019年-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>機械学習の普及が加速する2019年</a></span></li></ul></li></ul></div>

# 概要
機械学習について調べた履歴を記録する。  
Jupyter Notebook の便利な機能は、履歴作成に当たって大変助かる。(Table of Contents の自動生成など)
まずは、細かい整理はせず、片っ端から記録するというスタイルで始めるとしよう。  


# 2018-12-18

## 検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ
  https://recruit-tech.co.jp/blog/2018/09/10/qass_ml/
  
* 機械学習システムのアーキテクチャ図があるので参考になる。
    * ![アーキテクチャ](https://s3-ap-northeast-1.amazonaws.com/prod-rtc-blog/wp-content/uploads/2018/08/31193348/flya.png)
* ソフトウェア資産の再利用性を高めるため、Dockerを活用している。
    * Google Container Registry (GCR)に、コンテナを登録している。
* Google Kubernetes Engine (GKE)の上に、"DRONE", "DigDag" といったアプリケーションを構築。
* 機械学習処理の最終出力は、Google Cloud Storage(GCS)上に保存。
* 処理フローの共通化のポイント
    1. モデル部分は共通化しているが、
    2. 前処理部分は、サービスごとに開発する
    
### 登場キーワードについての調査

#### DRONE : CI/CD のパイプライン処理

JenkinsやCircle CIと同じカテゴリのCI/CDツールだが、**Gitイベント** をトリガーにして、**Dockerコンテナ**内部でジョブを実行することに特化したツールである。  
Dockerを利用したマイクロサービス化が普及するにつれて、DRONEの普及も進んでいる模様。  

1. 対象Gitリポジトリのルートディレクトリにある **.drone.yml** を読み込んで、パイプライン処理を実行する。
   .drone.ymlの例
   ***
```
pipeline:
  # ステップは上から順に実行されます（順序付きmap）。
  hello:
    # ステップ毎にイメージを指定
    image: alpine:3.6
    # Dockerコンテナ起動時のエントリポイントとしてコマンドが実行されます。
    # 各コマンドは同じシェルで実行されます。
    commands:
      - echo 　"Hello World"
```
   ***

参照: https://engineering.linecorp.com/ja/blog/go-oss-ci-tool-drone-replaces-jenkins/


#### DigDag : 機械学習処理システム向けのパイプライン処理ツール

機械学習処理システムは、処理フローが複雑化しやすく（特に前処理）、技術的負債の利子が危険レベルになる傾向がある。  
処理フローを記述しやすく、把握しやすく管理できるのがDigDagとのこと。  
以下のように、処理フローを階層的に記述しやすい。

![DigDag処理フローイメージ](http://docs.digdag.io/_images/grouping-tasks.png)


yml的な記述をする .dig ファイルに処理フローの定義を記述する。

.dig の例
***
```text
# daily_etl.dig
schedule:
  daily>: 00:05:00
 
+foo:
  _retry: 3
 
  +get_foo_metrics1:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics1 1day > foo_metrics1_daily.csv
  +vaidate_foo_metrics1:
    sh>: bash data_validator.sh foo_metrics1_daily.csv
  +commit_foo_metrics1:
    sh>: bash commit.sh foo_metrics1_daily.csv
  +get_foo_metrics2:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics2 1day > foo_metrics2_daily.csv
  +vaidate_foo_metrics2:
    sh>: bash data_validator.sh foo_metrics2_daily.csv
  +commit_foo_metrics2:
    sh>: bash commit.sh foo_metrics2_daily.csv

```
***

学習処理を定義する .dig ファイルの例を示す。
***
**daily_train.dig**
```
# daily_train.dig
schedule:
  daily>: 00:10:00
 
+wait_daily_etl:
  require>: daily_etl
 
+foo:
  +train:
    _parallel: true
 
    +train_foo_metrics1:
      sh>: python train.py foo metrics1 foo_metrics1_daily.csv
    +train_foo_metrics2:
      sh>: python train.py foo metrics2 foo_metrics2_daily.csv
 
  +cross_validation:
    _pararell: true
 
    +cross_validation_foo_metrics1:
      sh>: python cross_validation.py foo metrics1
    +cross_validation_foor_metrics2:
      sh>: python cross_validation.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

cross_validation.py は、過学習を起こしていたと判断すると、終了コード 1 を返すプログラムとなっている。  
過学習が起きると \_error が呼び出され、通知処理(notify.sh)が起動するようになっている。  


週次の処理をする場合は、以下のような依存関係を前提とした .digファイルを記述する。  
以下の.digファイルは、 requireで指定された daily_train.dig が7回実行されるのを待ってから、foo 以下の処理が実行される。

***
```
# fine_tuning.dig
schedule:
  weekly>: Mon,01:05:00
 
+wait_daily_train:
  loop>: 7
  _do:
    require>: daily_train
    session_time: ${moment(last_session_time).add(i, 'day').format()}
 
+foo:
  +train:
    _parallel: true
 
    +fine_tuning_foo_metrics1:
      sh>: python fine_tuning.py foo metrics1
    +fine_tuning_foo_metrics2:
      sh>: python fine_tuning.py foo metrics2
```
***


定期的にポーリングして、モデルの性能悪化を検出したら、すかさず通知したいときには、以下のような .digファイルを作ると良いという。  
以下では5分ごとに predict.py を実行し、モデルの性能が悪化して exit_code = 1 となったら、_error がキックされ、通知処理(notify.sh)が実行される
。  

*** 
```
# predict.dig
schedule:
  minutes_interval>: 5
 
+foo:
  +predict:
    _parallel: true
 
    +predict_foo_metrics1:
      sh>: python predict.py foo metrics1
    +predict_foo_metrics2:
      sh>: python predict.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

#### .dig ファイルのサンプル集

トレジャーデータの以下のサイトで紹介している。  
https://github.com/treasure-data/digdag/tree/master/examples

##### 参照リンク
* [DigDag公式サイト](http://docs.digdag.io/index.html)
* https://www.lifull.blog/entry/2017/09/20/151600

# 2018-12-19

## 機械学習プロジェクトが失敗する9つの理由 - 六本木で働くデータサイエンティストのブログ
出展  https://tjo.hatenablog.com/entry/2018/08/03/080000

### ML Ops のポイント

DevOpsならぬ **"ML Ops"** が普及している。ML Ops で関心があるとされているのは以下の４点。  
1. システムやサービスのどの場所で機械学習を使うのか
2. 機械学習システムの運用方法のポイント
    * 機械学習処理を含むコードのテスト方法
    * 構築したモデルのデプロイ方法
    * モデルとデータの監視
        * 性能が悪化したことを検知して、モデル更新のタイミングを知らせる方法などか？
    * 開発～デプロイプロセスの自動化の方法
        * 先に触れたリクルートテクノロジーズのエントリーが該当するかもしれない
3. 機械学習処理システムの高速化(ロジックのチューニングやGPUの利用など)
4. 機械学習処理システムの開発体制,運用体制

### 機械学習プロジェクトが失敗する理由

#### 課題設定の誤り
不正検知の場合、「過去に人が見つけてきた事例」をベースに不正を定義してしまうと、**「新しい手口の不正」**に対処できなくなってしまう。  
この場合は、**「何が正常なのか」**を学習させるのが適切である。  
異常時のデータが入手困難であれば、大量の正常データから、正常の定義をしっかりと構築し、そこから逸脱する異常値を検知するというわけである。  
> 例えば「正常値データで学習させたAutoencoderで入力データを復元できたら正常値、復元できなかったら異常値」というやり方などは好例かと。

#### 間違った問題に機械学習を使っている
費用対効果を考慮せずに、闇雲に機械学習システムを作ると大変なことになる。  
明確なニーズがあること(=機械学習を用いて解決するだけの**ビジネス的価値が高い問題**であること)を確かめずに、高いコストを払って機械学習システムを構築すると大赤字である。   
> 「2000万円の人件費を浮かせるために1億円かけて機械学習システムを作ったのに、売上高はたったの3000万円」とかだと目も当てられません。

#### 十分なデータがない
そもそも**「データが入手困難/不可能な分野」**に、機械学習システムを投入することが戦略的に誤りである。  
どんなに優秀な重火器（優秀なモデル）をそろえても、弾薬（データ）が無ければ戦うことはできない。


#### 適切なデータがない（データのクオリティが低い）
まさに**"Garbage in, garbage out"**。データの量はあるが、役に立たないか、誤ったデータばかりだと、モデルがいくら優秀でも誤った判断に収束してしまう。  
データ有効性チェックプロセス(Data Validation Process)のための専任部署を立ち上げる企業もあるという。  
ビジネス的価値の高い問題に取り組むのであれば、そうした専門部署を立ち上げても十分割に合うのではないか。

#### (不要な）データが多すぎる
データの量はあっても、手がかりとならないデータばかりだと苦労する。  
「特徴量削減」の苦労である。  
PCAやt-SNEで次元を圧縮するにせよ、特徴量を削減するにせよ、どの特徴量を残すのかは人の手にゆだねられる。  
データを蓄積する段階から、機械学習にかけることを考慮した設計にしておくことが望ましい。  

#### 人材の配置が適材適所ではない
チームが小さい、部門の立ち上げ時期には、データ収集・加工・前処理・仮説検証・モデル構築・プロダクト実装・デプロイまで出来るオールマイティな古巣タックな機械学習開発者/データサイエンティストが数名いればよい。 

だが、チームが大きくなってきたら、ETL特化型、モデル構築特化型など、機械学習分野の中でも特化した人材がいると良い。  
データ基盤を世話するデータアーキテクトは、部門や事業の規模が大きくなってきたら必須になるだろう。  

#### 間違ったツールを使っている
レガシーな既存システムに縛られ、規模がスケールしにくい、従来の技術やシステムに固執するのは危険である。  
将来、高い確率で**技術的負債**になる。  
なるべく新しく、**スケールする技術**を取り入れるべきである。  
* オンプレのデータベースよりは、クラウドのデータウェアハウス(GCPのBigQueryとか)

#### 適切なモデルを使っていない
闇雲にディープラーニングを導入するのは、問題である。適切なチューニングが出来ないと目も当てられなくなる。    
例えば、線形分離可能な問題には線形モデルを適用すべきである。非線形モデルを入れるとおかしくなる。  
機械学習モデルも適材適所である。

#### 適切な評価指標を使っていない
予測する対象、課題に応じて、重視する評価指標を使い分ける必要がある。  
異常検知の場合、大多数が正常なので、精度(accuracy)で言えば常に１００％になってしまう。
実際には、見逃すとリスクが大きいので、対象精度を犠牲にしてでも高い再現率(recall)を達成すべきケースが多いだろう。  




# 2018-12-25

## データサイエンティスト・機械学習エンジニアは常に最前線にいるべき理由

お題となるエントリーはこちらである。  
***
[データサイエンティストや機械学習エンジニアが、可能な限り統計学や機械学習やプログラミングを使って課題を解決するべき3つの理由 - 六本木で働くデータサイエンティストのブログ](https://tjo.hatenablog.com/entry/2018/12/04/190000)
***

### 自分のキャリアと会社を切り離して考えるべき時代
> 私の意見では、個人のキャリアは会社とは切り離して考えるべきであり、従業員は経営者視点（会社が儲かるなら何でもやる）になってはいけない。会社や顧客のために個人のキャリアや生き方（データ分析以外の仕事も何でもやるデータ分析者）を犠牲にするような労働はすべきではない。

データ分析の現場から離れて、パワポ作りや営業的タスクに傾いてゆくと「**おしゃべり課題解決コンサルおじさん**」になってしまうという主張は、尤もである。  
最新の技術や理論にキャッチアップできなくなると、コンサルタントの一人として埋もれてしまうのは容易に想像できる。  

また一つの会社に人生をささげる時代ではなくなり、ポータブルスキル・アビリティーというべき、どこに行っても認識されるし、通用するだけの独立した能力や技能・知見を養っていくべき時代が訪れつつあるような気がしている。  


### データサイエンティスト・機械学習エンジニアというキャリアの信用と独自性を養う

#### データサイエンティスト・機械学習エンジニア像（ざっくり＠２０１８）
> これらの職業はそれぞれ「統計分析」and/or「機械学習」そしてプログラミングで高度に自動化されたプロセスに基づくデータ分析を担う、独自性のある専門職として認識されるようになっていった、というのが僕個人の見解です。

本来的には、統計分析・機械学習に通じており、Python等のプログラミング言語を用いて、データ解析・機械学習システムを構築する能力・経験・知見を持つのが、これらデータサイエンティスト・機械学習エンジニアの姿というべきだろう。  
機械学習エンジニアは、仮説・モデル構築というよりは、構築されたモデル（検証済み）を、実稼動するプロダクトとして落とし込むという、ソフトウェアエンジニアリングよりの役割が、次第に鮮明に求められるようになっている模様。  
分業化が始まっているようだ。（特に、データ分析部門、AI部門の歴史が長い大手IT企業）

#### なんちゃってデータサイエンティスト
> 必然的に統計学も機械学習も況してやプログラミングも出来ない「なんちゃってデータサイエンティスト」が続出し、彼らがレベルの低いアウトプットを繰り返して社会からの幻滅を誘ったことで、第一次ブームは萎んでいったのでした。

なんちゃってデータサイエンティスト、「でーたさいえんてぃすと」というキーワードが、具体的にどういった人物像を示しているのか今まで判然としなかったが、これで具体的になった。  
ビジネス畑出身で、統計学・機械学習・プログラミングのいずれのバックグランドを持たず、分析業務に手を出して、いたずらに周囲を混乱させた人物ということだろう。  
確かに、そうした人物がデータサイエンティストであったら、どうしようもない。  


#### Excelに固執せず、スケールするテクノロジーを
> 「統計学や機械学習やプログラミングを使わずに問題を解決してしまうと、次からそれらを使える機会が失われやすい」
> 前述したように、データサイエンティストや機械学習エンジニアの独自性は「統計学や機械学習やプログラミングを用いて問題を解決できる」点にあります。にもかかわらず、それらを使える機会を失うということはそれらの職業としての独自性を失うということです。

Pythonではなく、Excelベースでデータ分析をさせる会社は危険である。  
扱うデータの規模が拡大した際に、手の打ちようがない。スケール困難である。  
よって、Python、Hadoop+Hive、BigQuery、AWS などスケールが見込めるテクノロジーを用いて、分析基盤や分析プロセスを確立しておくべきであろう。  
Excelに固執する会社は、テクノロジーがもたらすデジタルトランスフォーメーションの波への適応を拒否しているも同じであり、淘汰の運命を受け入れたのに等しいだろう。  

こうしたテクノロジーに立脚した仕事の進め方に軸足を置くことは、データサイエンティストや機械学習エンジニアというキャリアの信用を養い、かつ自分自身を守る上でも、実に合理的であろう。（さらに、データ解析を導入する企業にとっても長期的展望にたって有益である）


### データサイエンス・機械学習導入の意義

* 人の直感的認識を越えた複雑な認識ができるようになる。

> 「属人性」を排し、「再現性」高いデータ分析を行い、「自動化」によって、「スケーラビリティ」の向上を目指す

担当者がいなくなっても大丈夫、安定して再現性が高い分析ができるし、自動化されているので取り扱うデータ量が増えても対応できる（＝スケーラビリティ）


### 時代の流れに反しない仕事のやり方を

> 統計学や機械学習やプログラミングによって課題解決していくことが「好き」な人が、ビジネス上の要請だからという理由でそれらをあえて封印するような仕事のやり方をするべきではない、

データに基づいた意思決定プロセスが、世の中に浸透・普及しつつあるこの時代に、前時代的なアプローチに戻ることは危険な逆走行為であろう。  
古いやり方に固執するのは、基本的には避けるべきである。  
ただし、競争力の源泉となるような本質的な要素であり、AIやデータ分析ではまだ代替できないほど高度なものであれば、尊重すべし。  
対人コミュニケーションや高度な意思決定、創造的プロセスなど、AIではまだ到達できていない高度なものは残るだろう。

それ以外、手順は明確、自動化の可能性が十分あるのに、変化を拒否したいがためだけに、時代の流れに逆行するスタンスを取るのは完全にＮＧだろう。  
無自覚な自殺行為であり、それを指摘し、変えなければ生存が危うくなるだろう。  


### メモ

* 「多変量モデル」＝特徴量を使って、目的変数を予測する、つまり機械学習モデルは多変量モデルに含まれる

> ということで第二次データサイエンティストブームも間も無く崩壊するのだろうなと思っています。同時に人工知能ブーム、そして機械学習エンジニアブームも崩壊するのか否かを、今後は注意深く見守っていく必要がありそうです。。。

このあたりはポジショントークである。分かりやすい。  
データサイエンティスト、機械学習エンジニアの希少性を維持し、陳腐化を防ぐため、やんわりと超間接的にアプローチをしているといったところか。  


# 2018-12-28
## 年末に勉強しておきたいAIのAPI

年末年始にAIを使い倒す。一度は試したいAPI&ライブラリまとめ
https://ledge.ai/ai-library-api-2018/

年末年始だけで、使い倒しきることができるのだろうか？と不安になるほど、バリエーションが多い。  
使ってみる、使用体験を蓄積するぐらいなら、なんとかなるのではないか？


## 最先端と現場の狭間で

お題のエントリはこちら。約1年前のエントリである。
https://tjo.hatenablog.com/entry/2017/12/27/190000#fn-31d5211b


### 怒涛の勢いで発達する最先端

深層学習分野の発達速度は、とんでもないことになっている。  
プロのデータサイエンティストでさえ、キャッチアップを放棄するほどとは。  
効率的にキャッチアップするには、キーワードと概要だけ覚えておき、必要なときに実装コードをGitHubで探すというもののようだ。  
たしかに納得感がある。論文執筆者が実装コードを提供するケースは、今後増えるのではないかと体感しているので、
とりあえず気になるものがあったら、GitHubでコードを落としてきて、Kaggleのカーネルで動かしてみるか。(GPUが使える！)

昨年末時点での先端キーワードだが、
* AutoML : https://research.googleblog.com/2017/11/automl-for-large-scale-image.html  
  パラメータはともかく、ネットワーク構造まで自己学習とは、末恐ろしい。
  > AutoMLのような「最適なモデル（ネットワーク＆パラメータ）自体をそもそも自己学習によって作り出す」仕組みまでもが出てくるようになり、  
  
  
* Deep Image Prior : https://github.com/DmitryUlyanov/deep-image-prior
  大量のデータを必要としない深層学習とは、、、
  > 「そもそも大量の学習データを必要とせず（それどころか単一の画像のみから）ノイズ除去・欠損部の補完を行う」という代物で、ここまで来ると機械「学習」とは一体何だったのか感すら覚えます。
  
* interpretability（解釈性）: https://christophm.github.io/interpretable-ml-book/
  さすがにそろそろブラックボックス性に対する不安の高まりに対処すべきときがきたか
  > これだけpredictivity（予測性能）の方に先鋭化してきている機械学習に対して、interpretability（解釈性）について考えようという動きも出てきているようです
  
  

### 機械学習の現場

機械学習のコードの実装が占める割合のイメージがこれである。  
確かに自分の感覚に近い。大きなズレはない。

![機械学習の現場の仕事のマップ](https://cdn-ak.f.st-hatena.com/images/fotolife/T/TJO/20171208/20171208145454.png)


データを集める、データを加工する、特徴量を考える・・・機械学習モデルに投入するのは終盤である。  
最後の総仕上げに近いものがある。(最も、性能によっては、作業のサイクルを繰り返すのだが）  

機械学習に投入するデータを格納するインフラ、格納するデータの仕様、データにアクセスするための機能、システムの開発、  
実際の作業では重要度が大きいのはこちらか。  
機械学習のモデル周辺の作業しか出来ないと、現場の作業にあまり貢献ができなくなってしまうだろう。  
機械学習専攻の新卒が、現場のギャップで苦しむのは、確かにこういう泥臭いところかもしれない。  
モデルに投入するのは、最後の総仕上げ、という感覚が、理論との現実の間のギャップの源泉になっているかもしれない。  



# 2018-12-30
## 便利なリンク集

### 統計用語集
良く使う統計用語集が説明サマリと共にまとめられている。  
学習範囲の把握に便利だろう。というか、ここをハンドブック的に見るだけでだいぶ便利。  
https://home.hiroshima-u.ac.jp/mfukuok/er/Etc_TKG(4).html

### 確率分布一覧
統計で登場する様々な確率分布が一堂に会している。  
探索的データ解析の際に、良く参照するのではないか。  
https://www.biwako.shiga-u.ac.jp/sensei/mnaka/ut/statdist.html

#### ガンマ分布
ガンマ分布のコンセプトの解説とRによるサンプルコードとグラフが乗っている。  
http://yanbow221.hatenablog.com/entry/2017/09/06/211456

# 2018-12-31
## 2019年のスタートアップを占う
TechCrunchより。  
https://jp.techcrunch.com/2018/12/28/2019-statup-trend/ 

* ソフトウェアが世界を塗り替える
> 日本について言えば、Software is eating the world的な文脈での、既存産業の構造転換、SaaSへの置き換えです。
> その他の業界もジリジリとソフトウェアに「食われていく」と考えてます。

* 金融方面
> 既存金融が十分に提供できない残りの巨大な領域、オルタナティブ・ファイナンスは全般的に面白い。小口の融資全般、ファクタリング（請求書の売却）、ABL（動産担保ローン）など、与信モデルの発明や既存ユーザベース活用によって参入余地が多く生まれるモデル。

* 既存インターネット産業のレガシー化とそのリプレイス
> 古いタイプのインターネットビジネスをリプレイスするというのも大きなテーマになると思います。最近だと、投資先ではありますが、コスメのコマースメディアを展開するnoin社は急成長していますね。

* 人の能力拡張
> バイオテックとITがより結びついていく世界ですね。この領域は今後の10年間で大きく発展すると思うので、かなり注目していますね。

* 伝統的産業、レガシー産業の急激なIT化（主にSaas)
> 2018年は「産業SaaS革命」が大きく進んだ年。様々な古い産業に向けたSaaSスタートアップが目立ちました。例えば、建設業界のANDPAD、食品工場向けのKAMINASHI、保険はHokan、カスタマーサポートはKarakuri、歯科医はDentalight、薬局はKakehashiなど。

* 既存業界に切り込むスタートアップ
> 2018年にも建設業界・保険業界・物流業界といったところに変革を起こすであろうスタートアップが数多く生まれ、そして資金調達に成功しています。

* B向け事業支援型Saasの活躍
> エンタープライズ分野では引き続きDigital Marketing（フロムスクラッチ、サイカ、toBeマーケティング）、Sales Tech（UPWARD、マツリカ）、HR Tech（チームスピリット、Refcome）を中心に、カスタマーサクセスを実現するSaaSベンチャーへの投資を増やしたい。

* 消費者向けサービスのアップデートがまだ進む
> 2018年で印象的だったのは“新世代のコンシューマーニーズ”でした。スマホシフトが済んでコンシューマーサービスは一旦掘り尽くしたとおっさん世代は思っていた気がします。しかし、ここにきてTikTok、Vtuber、ライブ系、インフルエンサー系などなど新基軸のコンシューマーサービスでZ世代の支持を得てヒットしたのが結構ありました。このトレンドを読めなかったのは痛恨の反省です。2019年も文化を創るようなコンシューマーサービスは引き続き続くと思いますし、投資対象として注目していきたいです。

* 2019年の景気低迷を受けて傾向が変化する分野
    * 落ち込む分野
    > 4K（広告費、交際費、交通費、研修費）＋人材採用市場、個人向け高額娯楽（宿泊費、飲食費）
    
    * 伸びる分野
    > クラウドなどのB向けコストダウン（生産性向上）、ゲームなどの家庭内少額個人課金、自己啓発への投資
    
* Biotech is eating the world
> 2018年の米国IPO市場においてバイオ銘柄は過去最高の盛り上がりを見せ(58社上場、US$6.3Bの調達)、全体の約30%を占めるまでになりました。
特にメッセンジャーRNAによる治療薬を開発しているModerna（約5000億円）や老化細胞の治療法を開発するUNITY Biotechnology（約700億円）、次世代ゲノム編集技術のHomology Medicines（約700億円）などユニークな企業が上場を果たしました。2019年は特にユニコーンに代表されるSamumed（1兆2000億円）など、Longevity／Anti-agingが大きなテーマになるのではと考えています。

* Maas(移動手段の革命?)
> LyftやCREWなどのライドシェアについては、コミュニケーションや決済面に関し、これまで苦痛だったタクシーのUXを凌駕しています。
> 個人的にはバス版Uberと言われるChariotのような、複数人のバス乗車希望位置を解析した走行ルート提示サービスの出現に期待しています。

* D2Cの発達による中間業者スキップがさらに進む？
> ユーザーとの関係構築が容易になり、中間事業者を介する必要がなくなってきたため、ファッションやコスメ等ブランドを持つ生産者と消費者が直接取引するD2Cの流れがさらに加速しそう。

* VR世界の発達
> 4度目のVR元年である2019年は、Oculus Questをきっかけにブレイクします。その上でVtuberによる本格的なライブ、VR上でのアバタービジネス、VR上でのコミュニケーションなど、今後10年の基礎となる体験をする人が増えてくるのではないかと思っています。

* 大企業のベンチャー化？
> 今までベンチャーに投資してこなかったような重厚長大系の企業がMaaS/コネクテッドカー、航空宇宙、FinTech、バイオ、AI、ビッグデータなどの分野で、アーリーからレイターまで様々なステージで、本気の資本業務提携に乗り出し、一緒に事業開発を進めるケースが目立ちました。

# 2019-01-07

## おさらい：昨年話題になったテクノロジートピック@機械学習分野

参考エントリ：https://tjo.hatenablog.com/entry/2018/12/25/190000


### Google Research BERT

> メディアやSNSの話題をさらったものとして何か一つ研究を挙げるとしたら、BERTでしょうか。「自然言語理解タスクでヒトの精度を超えた」などと喧伝されて大きな話題を呼びました。

arXiv の論文は[こちら](https://arxiv.org/abs/1810.04805)
BERTとは、Bidirectional Encoder Representations from Transformers(変換器由来の双方向符号表現？)の略である。  

GitHub で[実装コード](https://github.com/google-research/bert)が公開されている。  
解読して使いこなすようになるのに1ヶ月ぐらいか。  
話題に上がったモジュールやアルゴリズムの学習コストは、平均1ヶ月として取り組んでみるか。

Google検索やQuiitaで検索したところ、実装例やエントリを見つけることができた。  
日本のデータサイエンティストの間でも既に有名であった。  
最新の機械学習分野のエントリをキャッチするRSSリーダーの準備待ったなし。（あとツイッターも）

#### BERT関連情報リソース
* [汎用言語表現モデルBERTを日本語で動かす(PyTorch)](https://qiita.com/Kosuke-Szk/items/4b74b5cce84f423b7125)
* [BERT:自然言語処理のための最先端の事前トレーニング](https://webbigdata.jp/ai/post-2132)
* [BERT with SentencePiece で日本語専用の pre-trained モデルを学習し、それを基にタスクを解く - クックパッド開発者ブログ](https://techlife.cookpad.com/entry/2018/12/04/093000)
* [PyTorch版 BERT](https://github.com/codertimo/BERT-pytorch)
* [[翻訳]BERTで自然言語AIをはじめる（github上のREADMEの翻訳） - Qiita](https://qiita.com/uedake722/items/927bf491a025f1a88b17)
* 便利[実装例あり：5分でできる：Googleの自然言語処理AI（BERT）をTPU上で転移学習 - Qiita](https://qiita.com/uedake722/items/fb9877fc45224353b44b)
* [BERT多言語モデルで日本語文章の二値分類を試す - Qiita](https://qiita.com/knok/items/9e3b4505d6b8f813943d)

### Deep Learningはなぜ大域最適解にたどり着けるのか

> 今でもよく話題に上る「何故Deep Learningはどれほど複雑で非線形な課題であっても大体最適解にたどり着いてしまうのか？」という疑問の一端に答える研究だと思いました。

arXiv の論文は[こちら](https://arxiv.org/abs/1802.04474)。
> We theoretically discuss why deep neural networks (DNNs) performs better than other models in some cases by investigating statistical properties of DNNs for non-smooth functions. While DNNs have empirically shown higher performance than other standard methods, understanding its mechanism is still a challenging problem. 

> From an aspect of the statistical theory, it is known many standard methods attain the optimal rate of generalization errors for smooth functions in large sample asymptotics, and thus it has not been straightforward to find theoretical advantages of DNNs. This paper fills this gap by considering learning of a certain class of non-smooth functions, which was not covered by the previous theory. 

> We derive the generalization error of estimators by DNNs with a ReLU activation, and show that convergence rates of the generalization by DNNs are almost optimal to estimate the non-smooth functions, while some of the popular models do not attain the optimal rate. In addition, our theoretical result provides guidelines for selecting an appropriate number of layers and edges of DNNs. We provide numerical experiments to support the theoretical results.

不連続関数の予測にDeep Learningは強く、さらに適切なレイヤー数や辺（エッジ）数を推定することもできるという。  

スライドは[こちら](https://www.slideshare.net/masaakiimaizumi1/ss-87969960/)
<iframe src="//www.slideshare.net/slideshow/embed_code/key/9flZRKN0VkALc4" width="595" height="485" frameborder="0" marginwidth="0" marginheight="0" scrolling="no" style="border:1px solid #CCC; border-width:1px; margin-bottom:5px; max-width: 100%;" allowfullscreen> </iframe> <div style="margin-bottom:5px"> <strong> <a href="//www.slideshare.net/masaakiimaizumi1/ss-87969960" title="深層学習による非滑らかな関数の推定" target="_blank">深層学習による非滑らかな関数の推定</a> </strong> from <strong><a href="https://www.slideshare.net/masaakiimaizumi1" target="_blank">Masaaki Imaizumi</a></strong> </div>

* 予測対象が、連続な関数（＝微分可能な関数）の場合、既存のモデルでも最適化可能。


### AutoML

データサイエンティストが手間隙掛けて構築した高精度モデルに、自動構築モデルが性能で肉薄するという。  
なんという時代が来たのだろうか。
> リンク先の記事のラーメン二郎分類器の事例は、非常にインパクトがあって面白かったです。発案者の人が手動で細かくチューニングして達成した高い精度に、AutoMLだけで迫ることが出来たというのは色々な意味で印象深いです。

https://cloud.google.com/blog/products/gcp/automl-vision-in-action-from-ramen-to-branded-goods

メルカリは、AutoMLを使って出展商品の自動認識システムの構築に挑戦している模様。  
> AutoML Vision also proved its ability in quite a different use case: classifying branded goods. Mercari, one of the most popular selling apps in Japan (and also gaining in popularity in the U.S.), has been trying out AutoML Vision for classifying images with its brand names.
![メルカリAutoML](https://storage.googleapis.com/gweb-cloudblog-publish/original_images/automl-ramen-mercari-5n32h.max-2800x2800.PNG)


#### ポイント：機械学習システムのマッシュアップ？

データさえ用意すれば、APIを使って高精度の認識モデルを自動的に構築できる時代になったということか。  
AutoMLを使いこなすデータサイエンティストに価値が出てくるということかもしれない。  
(データの用意の仕方、AutoMLの組み込み方などなど）


### 転移学習の使いどころ

あらかじめ学習しておいたモデル(pre-trained model)を使い、入力データをベクトル化し、アプリケーションを開発できるという。  
> pre-trained NNLMを使えばその全てをスキップして一発で特定の次元数の特徴量に変換してくれるからです。必要な前処理もたかだか分かち書きぐらいで、それでもかなり高い分類精度を実現してくれます。

> 他にもInception-v3のような巨大なpre-trained modelを使うことで、ImageNet Challengeで途方も無い高精度を達成した機械学習モデルの「叡智」をそのまま転用して、新たにカスタマイズした画像認識モデルを作るという芸当も出来るわけです。

出来上がっているモデルの全結合層の手前までを使い、入力データを効率的にベクトル化（一種の構造化）ができることは非常に大きなメリットになる模様。  

既存モデルの流用スキルは、今後データサイエンティストの間で重宝されるかもしれない。




## DeNAのKaggle制度

話題だけ知っていたが、実際の制度設計を見たところ、Kaggle Master以上を前提とした制度になっていた。  
これは確かにレベルが高い。世界的に見てもトップレベルのデータサイエンティストを集めるという意思の表れということか。  
![DeNAのKaggle制度](https://www.sbbit.jp/article/image/35547/l_bit201810051615570812.jpg)


## 機械学習の普及が加速する2019年

参考:http://ascii.jp/elem/000/001/673/1673587/

> その結果、筆者としては、ここ数年続いた「AIブーム」がいよいよ終わりを告げることを確信した。
> 今年のSXSWでの人工知能の活用事例は、十分に確立された、いわば「枯れた」技術を導入するという、地に足のついた着実な試みが多かった。  
> ・・・中略・・・  
> 異なる業界に関する記事に含まれるテキスト情報を「掛け合わせ」して、共通して出現する単語を抽出し発想のヒントを得る「画期的な」ツールなどが紹介された。
>　しかし、このツールを構成する要素技術として使われていたのは、2013年頃にテキスト解析関連の研究者の間で一世を風靡し、現在ではほぼデファクト方式として定着している「word2vec」という、基本的なテキスト解析技術である。  
> ・・・中略・・・   
> SXSWのような尖った場でSalesforceのChief Scientistが人工知能の一般論を中心に話そうと判断したほど、人工知能が広く普及していく動きがいよいよ本格化してきており、ビジネス領域ではキャズム越えしつつあることを実感した。
  
  ブームは終わったというものの、それは機械学習や人工知能に対する失望から社会からそれらのテクノロジーが消えるという意味ではなく、実用段階に突入し、社会の隅々に浸透してゆくフェーズに入ったという意味の模様。  
  バズワードのフェーズから実用フェーズに進むというのでれば、それは大歓迎である。
  
> 今後、あらゆる業界において「人工知能」は静かに、当たり前のように使われるツールとなることは必然だが、いよいよ今、そのタイミングが来ているのである。

